In [2]:
from __future__ import print_function,division
import IPython
from IPython.display import Javascript,clear_output,display 
from ipywidgets import *
from ipywidgets import interact,interactive,fixed,interact_manual,Layout,jslink,Button,Layout,jslink,IntText,IntSlider,TwoByTwoLayout,IntSlider,Label,Text
import ipywidgets as widgets
import copy 
from bs4 import BeautifulSoup 
from urllib.request import Request, urlopen 
import pandas as pd 
import requests 
from io import StringIO 
from pychord import Chord

disable_js = """
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}
"""

In [3]:
# import song list
def song_table():
    orig_url='https://drive.google.com/file/d/1aDrlFu5eSabjo8CVKAOG6ljfr5OihZLq/view?usp=sharing'
    file_id = orig_url.split('/')[-2]
    dwn_url='https://drive.google.com/uc?export=download&id=' + file_id
    text = requests.get(dwn_url)
    text.encoding = 'utf-8'
    csv_raw = StringIO(text.text)
    df = pd.read_csv(csv_raw,encoding='utf-8')
    df['אמן'] = df['אמן'].apply(lambda x: x.strip())
    df['שיר'] = df['שיר'].apply(lambda x: x.strip())
    df['shir_aman'] = df['אמן']+" | "+df['שיר'] 
    return df

df = song_table()

In [4]:
def artist_songs(artist_name='א'):
      song_list = df.loc[df['shir_aman'].str.contains(artist_name)]['shir_aman'].sort_values()
      song_list = list(zip(song_list,song_list.index))
      return song_list

In [5]:
song_tab = widgets.Select(options=artist_songs(),rows=10,layout=Layout(width='auto'))

text = widgets.Text(value='',
                    placeholder='חפש שיר או אמן',
                    layout=Layout(width='auto'),
                    disabled=False)

def song_search_change(change):
    new_options = artist_songs(change.new)
    song_tab.options = new_options

text.observe(song_search_change, names='value')
#display(text,song_tab)

In [6]:
#song_list = df['shir_aman'].tolist()
#widgets.Combobox(
    # value='John',
 #   placeholder='Choose Someone',
  #  options=song_list,
   # description='Combobox:',
    #ensure_option=True,
    #disabled=False,
    #index = True
#)

In [8]:
button_LoadSong = widgets.Button(description="🤩 Get my Song 🎤",layout=Layout(width='auto'))
button_ChangeChords = widgets.Button(description="  Transform Chords  ",layout=Layout(width='auto'),)
output = widgets.Output()

progression_picker = widgets.Dropdown(options=[6,5,4,3,2,1,0,-1,-2,-3,-4,-5],value=0)
progression_picker.layout = Layout(width='8%',height='auto')

button_ChangeChords.button_style='info'
button_ChangeChords.style.button_color = '#D300E5'
button_ChangeChords.style.font_weight='bold'
button_ChangeChords.layout = Layout(width='91%',height='auto')

Chords_Hbox = widgets.HBox([button_ChangeChords, progression_picker])

song_tab.layout = Layout(width='auto',height='auto')
button_LoadSong.button_style='info'
button_LoadSong.style.button_color = '#5C9FF9'
button_LoadSong.style.font_weight='bold'
button_LoadSong.layout = Layout(width='auto',height='auto')
Song_Hbox = widgets.VBox([text, song_tab,button_LoadSong,Chords_Hbox])
Song_Hbox.layout.height = '300px'

song_show = widgets.HTML(value='',layout=Layout(width='auto',right='0px'))
song_show.layout.grid_auto_flow = 'row dense'
#song_show.layout.max_height = '200px'
#song_show.layout.grid_auto_columns = 'True'



display(Song_Hbox,song_show)

#display(button_LoadSong,button_ChangeChords,progression_picker,output)

Prog_value = progression_picker.value

def on_button_clicked(b):
    with output:
        #clear_output(True)
        #display(IPython.display.HTML("<marquee style='width: 30%; color: purple;'><b>✌ ...טוען שיר</b></marquee>"))
        #clear_output(True)
        progression_picker.value = 0
        song_show.value = "<marquee style='width: 100%; color: purple;'><b>✌ ...טוען שיר</b></marquee>"
        myurl = df.iloc[song_tab.value]['לינק']
        req = Request(myurl, headers={'User-Agent': 'Mozilla/5.0'})
        webpage = urlopen(req).read()
        soupified = BeautifulSoup(webpage,'lxml' ) # 'html.parser'
        song = soupified.find("div", {"class": "song_block"}) #Find the nevessary tag and class which it belongs to
        header = song.find(attrs={"class":"song_block_title"})
        # Clean the HTML from ads and more unnecessary stuff
        header = header.get_text().strip().replace("אקורדים לשיר ","")

        text_alignment = 'right'
        header = f"""<h1 style="text-align: {text_alignment}">"""+header+'</h1>'
        on_button_clicked.header = header

        progression_status =f"""<h4 style="text-align: {text_alignment}">"""+f"הסולם המקורי"+"</h4><h4></h4>"
        header_progression = header+progression_status

        delete_nodes = [{"class":"song_rating"},
                        {"class":"row"},
                        {"id":"songad"},
                        {"class":"song_block_title"},
                        {"id":"songTopAd8Td"},
                        {"class":"videosTitle"},
                        {"type":"text/javascript"},
                        {"id":"taboola-below-article-thumbnails"},
                        {"id":"downInSongTable"},
                        {"class":"chord_info"}]

        for node in delete_nodes:
            for div in song.find_all(attrs=node): 
                div.decompose()

        # Align guitar TABS to the left
        tab_list = ''
        tab_nodes = [{"class":"tabs"}]
        for node in tab_nodes:
            for div in song.find_all(attrs=node):
                div['dir'] = 'ltr'
                div['align'] = 'left'
                #div['display'] = 'inline-block'
                #div['width'] = '300px'
                div['position'] = 'absolute'
                div['right'] = '10%'
                #div['margin-left'] = '300px'
                tab_list=tab_list+'<p'+str(div)+'</p>'
                
        on_button_clicked.song=song
        # Merge Header & Song(words+chords)
        full_page=header_progression+str(song)
        on_button_clicked.full_page = full_page
        song_show.value = full_page
        #display(IPython.display.HTML(full_page))
        return on_button_clicked.header,on_button_clicked.song



def Chord_Progression(orig_chord=str,transpose_level=0):
    prog_chord = Chord(orig_chord)
    prog_chord.transpose(transpose_level)
    return prog_chord.chord

def on_button_clicked1(b):
    with output:
        clear_output(True)
        Prog_value = progression_picker.value
        Progression_song = copy.copy(on_button_clicked.song)
        header = copy.copy(on_button_clicked.header)
        for elm in Progression_song.find_all('span'):
            try:
                  elm.string = elm.text.replace(elm.text, Chord_Progression(orig_chord=elm.text,transpose_level=Prog_value))
            except:
                  dummy = 0
        text_alignment = 'right'
        if Prog_value > 0:
            progression_status = f"""<h4 style="text-align: {text_alignment}">"""+f"מודולציה: {Prog_value*0.5} טון למעלה"+"</h4><h4></h4>"
            header_progression = header+progression_status
        elif Prog_value < 0:
            progression_status = f"""<h4 style="text-align: {text_alignment}">"""+f"מודולציה: {Prog_value*0.5} טון למטה"+"</h4><h4></h4>"
            header_progression = header+progression_status
        else:
            progression_status =f"""<h4 style="text-align: {text_alignment}">"""+f"הסולם המקורי"+"</h4><h4></h4>"
            header_progression = header+progression_status
      # Merge Header & Song(words+chords)
        full_page=header_progression+str(Progression_song)
        on_button_clicked1.full_page = full_page
        song_show.value = full_page
        #display(IPython.display.HTML(full_page))
        return on_button_clicked1.full_page
        #print(full_page)
        

    
display(Javascript(disable_js))
button_LoadSong.on_click(on_button_clicked)
button_ChangeChords.on_click(on_button_clicked1)

HTML(value='', layout=Layout(grid_auto_flow='row dense', right='10px', width='auto'))

<IPython.core.display.Javascript object>